In [ ]:
!python -m spacy download ru_core_news_md

In [ ]:
import spacy
import pandas as pd

nlp = spacy.load('ru_core_news_md')

In [ ]:
topics = [69, 83, 124, 2, 60, 135, 166, 1, 90, 95, 96, 112, 117, 122, 161, 178, 183, 194, 213, 228, 91, 136, 157, 160, 19, 28, 30, 34, 37, 47, 55, 59, 63, 71, 73, 105, 125, 139, 151, 163, 177, 190, 198, 205, 211, 3, 5, 7, 8, 17, 22, 40, 46, 49, 65, 82, 87, 93, 114, 134, 140, 141, 146, 154, 162, 167, 169, 172, 173, 179, 184, 185, 195, 197, 199, 200, 202, 210, 212, 214, 217, 220, 222, 229, 36, 53, 67, 76, 84, 86, 120, 142, 143, 150, 152, 155, 156, 168, 191, 196, 227, 14, 57, 61, 104, 110, 159, 207, 6, 13, 38, 85, 92, 100, 149, 174, 216, 4, 10, 15, 21, 23, 27, 32, 39, 68, 107, 121, 132, 201, 231, 12, 33, 94, 99, 145, 181, 193, 11, 41, 180, 0, 25, 31, 64, 80, 106, 113, 118, 144, 158, 164, 175, 225, 230, 9, 16, 18]

In [ ]:
df = pd.read_csv('PATH/all_headlines.csv') # read all_headlines.csv

new_df = df[df["topic"].isin(topics)]
new_df

,Unnamed: 0,topic,document
0,207953,1,людей эвакуировать не стали в аэропорту кра...
1,51843,1,захарова о словах зеленского по эвакуации с ...
2,25631,1,красноярские железнодорожники волонтеры собрал...
3,63376,1,гуманитарную помощь доставили российские военн...
4,97112,1,тюменская область отправила партию гумпомощи ж...
...,...,...,...
248675,95779,231,грушко назвал формулировку о вступлении украин...
248676,184995,231,киевский эксперт отношения украины и грузии ...
248677,126562,231,тбилиси потребовал от киева доказать заявления...
248678,246036,231,мид грузии дипломатические миссии страны на с...


In [ ]:
new_df["document"] = new_df["document"].apply(lambda text: None if "новост" in text or "газет" in text else text) 
new_df = new_df.dropna()

In [ ]:
new_df["document"] = new_df["document"].apply(lambda text: " ".join(text.split()))
new_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,topic,document
0,207953,1,людей эвакуировать не стали в аэропорту красно...
1,51843,1,захарова о словах зеленского по эвакуации с аз...
3,63376,1,гуманитарную помощь доставили российские военн...
4,97112,1,тюменская область отправила партию гумпомощи ж...
5,131194,1,в общественных зданиях литвы решили организова...
...,...,...,...
248675,95779,231,грушко назвал формулировку о вступлении украин...
248676,184995,231,киевский эксперт отношения украины и грузии пр...
248677,126562,231,тбилиси потребовал от киева доказать заявления...
248678,246036,231,мид грузии дипломатические миссии страны на св...


In [ ]:
text = ". ".join(review for review in new_df.document)

In [ ]:
nlp = spacy.load("ru_core_news_md")
nlp.max_length = len(text)

In [ ]:
doc = nlp(text)
text_list = []
head_list = []

for token in doc:
    if token.is_alpha and not token.is_stop and not token.is_punct:
        if token.dep_ != "ROOT" and token.dep_ != "prep":
            head_list.append(token.head.lemma_)
            text_list.append(token.lemma_)

df = pd.DataFrame(list(zip(head_list, text_list )),columns =['head','text'])
combos = df.groupby(['head','text']).size().reset_index().rename(columns={0:'count'}).sort_values('count', ascending=False)



In [ ]:
combos

,head,text,count
331576,россия,против,3081
336181,санкция,россия,2612
53744,газ,российский,2469
74223,днр,лнр,2064
56696,глава,мид,1732
...,...,...,...
223256,перейти,топливо,1
38671,врач,елизавета,1
103347,защита,клиент,1
223252,перейти,таможня,1


In [ ]:
combos.to_csv("network_graph.csv")